In [ ]:
# Upgrade sqlite3 in colab to suppport window functions
!curl https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release | tar xz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.7/pysqlite3/_sqlite3.cpython-37m-x86_64-linux-gnu.so \
     /usr/lib/python3.7/lib-dynload/

In [ ]:
# After Restart runtime, run this cell
import sqlite3 as lite
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pandas.io.formats.info import DataFrameInfo
marketing_data = pd.read_csv('/content/drive/MyDrive/PMG/marketing_data.csv')
marketing_data.head()

,date,geo,impressions,clicks
0,2016/1/1,TX,2532,45
1,2016/1/1,CA,3425,63
2,2016/1/1,NY,3532,25
3,2016/1/1,MN,1342,784
4,2016/1/2,TX,3643,23


In [ ]:
store_revenue = pd.read_csv('/content/drive/MyDrive/PMG/store_revenue.csv')
store_revenue.head()

,date,brand_id,store_location,revenue
0,2016/1/1,1,United States-CA,100
1,2016/1/1,1,United States-TX,420
2,2016/1/1,1,United States-NY,142
3,2016/1/2,1,United States-CA,231
4,2016/1/2,1,United States-TX,2342


In [ ]:
conn = lite.connect('marketing_data.db')
cur = conn.cursor()
cur.execute("Drop TABLE IF EXISTS marketing_data")
cur.execute("CREATE TABLE marketing_data (date datetime, geo varchar(2), impressions float, clicks float )")
conn.commit()

marketing_data.to_sql('marketing_data', conn, if_exists='replace', index = False)


In [ ]:
conn1 = lite.connect('store_revenue.db')
cur = conn1.cursor()
cur.execute("Drop TABLE IF EXISTS store_revenue")
cur.execute("CREATE TABLE store_revenue (date datetime, brand_id int, store_location varchar(250), revenue float)")
conn1.commit()

store_revenue.to_sql('store_revenue', conn, if_exists='replace', index = False)

Question #0 (Already done for you as an example) Select the first 2 rows from the marketing data​

In [ ]:
# write your queries inside query as a string variable
query0 = '''
select * from marketing_data limit 2;
'''

# run script
pd.read_sql_query(query0,conn)

,date,geo,impressions,clicks
0,2016/1/1,TX,2532,45
1,2016/1/1,CA,3425,63


Question #1 Generate a query to get the sum of the clicks of the marketing data​

In [ ]:
#1 Generate a query to get the sum of the clicks of the marketing data​
query1 = '''
SELECT SUM(clicks) AS sum_clicks FROM marketing_data;
'''

# run script
pd.read_sql_query(query1,conn)
# The sum of the clicks of the marketing data is 1792

,sum_clicks
0,1792


Question #2 Generate a query to gather the sum of revenue by store_location from the store_revenue table​

In [ ]:
#2 Generate a query to gather the sum of revenue by store_location from the store_revenue table​
query2 = '''
SELECT SUM(revenue) AS sum_revenue,store_location FROM store_revenue GROUP BY store_location;
'''

# run script
pd.read_sql_query(query2,conn)

,sum_revenue,store_location
0,235237,United States-CA
1,51984,United States-NY
2,9629,United States-TX


Question #3 Merge these two datasets so we can see impressions, clicks, and revenue together by date and geo. Please ensure all records from each table are accounted for.​

In [ ]:
#3 Merge these two datasets so we can see impressions, clicks, and revenue together by date and geo. Please ensure all records from each table are accounted for.​
query3 = '''
SELECT * 
FROM store_revenue sr LEFT JOIN marketing_data md ON sr.date = md.date AND SUBSTRING(sr.store_location, -2, 2) = md.geo
UNION
SELECT * 
FROM store_revenue sr RIGHT JOIN marketing_data md ON sr.date = md.date AND SUBSTRING(sr.store_location, -2, 2) = m.geo;
'''

# run script
pd.read_sql_query(query3,conn)
# Does not support right join so just the query no display here

Question #4 In your opinion, what is the most efficient store and why?​

In [ ]:
#4 In your opinion, what is the most efficient store and why?​
query4 = '''
SELECT sr.store_location, sr.revenue, md.impressions INTO #temp
FROM store_revenue sr JOIN marketing_data md ON sr.date = md.date AND SUBSTRING(sr.store_location, -2, 2) = md.geo
)

SELECT store_location, 
		SUM(revenue) AS sum_revenue, 
		SUM(impressions) AS sum_impressions,
		ROUND(SUM(revenue)/SUM(impressions) * 1000, 2) AS efficiency
FROM #temp
GROUP BY store_location
ORDER BY efficiency DESC
LIMIT 1;
'''


Question #5 (Challenge) Generate a query to rank in order the top 10 revenue producing states​

In [ ]:
#5 (Challenge) Generate a query to rank in order the top 10 revenue producing states​
query5 = '''
SELECT store_location, 
        SUM(revenue) AS sum_revenue, 
        RANK() OVER (ORDER BY SUM(revenue) DESC) AS store_rank
FROM store_revenue
GROUP BY store_location
LIMIT 10;
'''